<a href="https://colab.research.google.com/github/kyeknu/Winter2025Lecture/blob/main/Copy_of_TEDA_1030_6_1_2_Missing_Values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 6.1.2 Missing Values

Missing values, also called *null values* indicates the absence of data. Null values can occur because of data that was inputting incorrectly or incompletely and can affect the analysis by skewing data incorrectly and giving incorrect insights. Null values also cannot be processed by machine learning models, which means that they cannot be present in order for machine learning to work.

Missing data can be treated with several different strategies, depending on the information available to the analyst and the importance of recovering the data.

1. Deletion: Rows with null values in a certain column or columns with many null values can be removed. However, removing rows and columns will also lead to a loss of data in those rows or columns. For example, in an analysis of grades in a classroom of 30 students, if only two assignments were turned in and were recorded as 100% and 98%, it may be misleading to say that the average score is 99% (although it is technically true).
2. Imputation: If other information can be used to fill in empty spaces, the information can simply be filled in, or imputed. This prevents a loss of data but is often difficult to do if the information is not contained somewhere else. For example, if an row (representing an order) is missing its final price but contains a unit cost and quantity, the total price could simply be computed and the blank space filled in.
3. Prediction: Machine learning models can be used to make predictions and fill in empty spaces. This is not as accurate as imputation but can help preserve data integrity in some analyses. For example, in an analysis of house prices, a model could be created that helps estimate the price of a house based on the number of bedrooms and bathrooms it has. This would not affect the measure of center for the houses but is not as accurate as just knowing what the actual price is. We will not touch on prediction in this class.



Let's learn to treat missing values by looking at a modified version of the Titanic data.

## About the data
This data set is a modified version of the classic Titanic data set. It contains many kinds of errors, which will be used to demonstrate cleaning concepts in this reading.

⚠ Warning: Remember to upload the Titanic Dirty data set to Google Colab before running the cells below.

In [ ]:
import pandas as pd
df = pd.read_csv('titanic_dirty.csv')
df.head()

## Identifying missing values

Right from the beginning, it's easy to see some null values in this data set in the `Age`, `Fare`, and `Cabin` columns. Null values are marked by `NaN`, meaning "not a number".

We can check how many null values there are in each column by using the `.info()` method on the dataframe.


In [ ]:
df.info()

This method gives us some important information. First, it says on the second line that there are 978 entries (rows) in total. Then, it lists each column and the number of non-null values that each one has. In the `Sex` column, for example, there are 819 non-null values, meaning that there are 159 null values in the column.

## Dropping null values

We can drop null values using the `.dropna()` method. Remember that the resulting dataframe will not be saved unless the `inplace=True` parameter is passed in.

In [ ]:
df.dropna() # Not saving the resulting dataframe on purpose

Observe the data set above. Thanks to the `.dropna()` method, all of the rows that had even a single null value were dropped. So how many rows do we have left?

Looking at the lower left corner of the dataframe above, we can see that after dropping all null values, we are left with only 144 rows. That's only 14% of the original data set!

The results above exemplify an important dilemma. By eliminating the null values in the data set, we've also eliminated a lot of good data too. So, how many null values should be cut out?

The answer is extremely subjective and dependent on the analysis. For an analysis that compares fares and ages, the `Cabin` column probably won't be very important. However, a lot of rows from above that actually had `Fare` and `Age` recorded correctly were dropped because they didn't list a `Cabin`. In that analysis, a better decision would probably be to drop the entire `Cabin` column first and then try to see how many null values were left.



## Dropping null values with a subset

Just like dropping duplicates, null values can be dropped using a subset of columns to look for null values in. For example, for performing an analysis of `Fare` and `Age`, null values in the `Cabin` column probably don't matter.

In [ ]:
df.dropna(subset=['Age', 'Fare'])

By only dropping rows that had a null value either in the Age or Fare column, 638 rows were conserved for analysis.

## Dropping specific columns/rows

In some cases, it may be necessary to remove a column from the dataframe. To do so, you can use the `.drop()` method and the `columns=` parameter, where columns is a list of column names to drop. Remember to use `inplace=True` to save the new dataframe.

In [ ]:
# Notice the `Cabin` column is gone
df.drop(columns=['Cabin'])

You can also drop specific rows from the dataframe using `.drop()` and the `index=` parameter, where index is a list of row numbers to drop. Dropping specific rows is a very uncommon task to perform, but could be done as follows:

In [ ]:
# Notice rows 1 and 3 are gone
df.drop(index=[1, 3])

## Imputation

Imputation is a subjective process in which empty values are computed using data that exists in the data set. For example, missing fares could be imputd using the median fare in the data set.

In [ ]:
# Get the median fare
df['Fare'].median()

When the value to be filled in is found, the `.fillna()` method can be used to fill in the null values of the column.

In [ ]:
# Fill in null Fares with the median fare -- notice row 4, which was previously a null fare!
df['Fare'] = df['Fare'].fillna(df['Fare'].median())
df.head()

You could also use a more complicated process to impute values. For example, notice that the `Sex` column contains 159 null values, but the `Name` column contains information regarding the passenger's `Sex` (Mr., Mrs., Rev., Miss). Because the information is contained in the data set, we can use it to impute the `Sex of each passenger.

In [ ]:
# Extract Mr., Rev., Mrs., or Miss using a regular expression and save to a new column
df['Title'] = df['Name'].str.extract('(Mr.|Mrs.|Rev.|Miss.)')[0]
df.head()

In [ ]:
# Use a for-loop to go through each row of the dataframe
# Remember that i represents the row number and row represents the row data

for i, row in df.iterrows():

    # If a title was found for the person and the title was 'Mr.'
    if row['Title'] == 'Mr.':
        # Change the `Sex` of that row to 'male'
        df.loc[i, 'Sex'] = 'male'

    # If a title was found for the person and the title was 'Mrs.'
    if row['Title'] == 'Mrs.':
        # Change the `Sex` of that row to 'female'
        df.loc[i, 'Sex'] = 'female'

    # If a title was found for the person and the title was 'Miss.'
    if row['Title'] == 'Miss.':
        # Change the `Sex` of that row to 'female'
        df.loc[i, 'Sex'] = 'female'

    # If a title was found for the person and the title was 'Mrs.'
    if row['Title'] == 'Rev.':
        # Change the `Sex` of that row to 'male'
        df.loc[i, 'Sex'] = 'male'

In [ ]:
df['Sex'].value_counts()

A for loop was used to go through all of the data, check to see if a title was found, and then change the `Sex`. Let's see how many null values are in that column now:

In [ ]:
df.info()

After performing that more complex process, there are 948 non-null values in the `Sex` column. That means that there are only 30 null values remaining. In other words, we were able to impute a large number of values in `Sex` column, but not all of them. We could keep perfecting the code an regular expression until we got a better result, if we wanted to.

This has been a short reading on fixing missing values. Missing values are common in data sets but are oftentimes fixable either by removing the values that are null or by imputing the values that are currently there.